In [1]:
import h2o4gpu
import h2o4gpu.util.import_data as io
import h2o4gpu.util.metrics as metrics
from tabulate import tabulate
import pandas as pd

In [2]:
"""
Import Data for H2O GPU Edition

This function will read in data and prepare it for H2O4GPU's GLM solver

Parameters
----------
data_path : str
             A path to a dataset (The dataset needs to be all numeric)
use_pandas : bool
              Indicate if Pandas should be used to parse
intercept : bool
              Indicate if intercept term is needed
valid_fraction : float
                  Percentage of dataset reserved for a validation set
classification : bool
                  Classification problem?
Returns
-------
If valid_fraction > 0 it will return the following:
    train_x: numpy array of train input variables
    train_y: numpy array of y variable
    valid_x: numpy array of valid input variables
    valid_y: numpy array of valid y variable
    family : string that would either be "logistic" if classification is set to True, otherwise "elasticnet"
If valid_fraction == 0 it will return the following:
    train_x: numpy array of train input variables
    train_y: numpy array of y variable
    family : string that would either be "logistic" if classification is set to True, otherwise "elasticnet"
"""

train_x,train_y,valid_x,valid_y,family=io.import_data(data_path="../../../data/credit.feather", 
                                                        use_pandas=False, 
                                                        intercept=True,
                                                        valid_fraction=0.2,
                                                        classification=True)

Reading Data with Feather
(23999, 25)
Original m=23999 n=24
Size of Train rows=19200 & valid rows=4799
Size of Train cols=24 valid cols=24
Size of Train cols=25 & valid cols=25 after adding intercept column


In [3]:
"""
Set up instance of H2O4GPU's GLM solver with default parameters

Need to pass in `family` to indicate problem type to solve
"""
print("Setting up solver")
model = h2o4gpu.GLM(family=family)

Setting up solver


In [4]:
"""
Fit GLM Solver
"""
print("Solving")
%time model.fit(train_x, train_y)
print("Done Solving")

Solving
CPU times: user 16.2 s, sys: 2.72 s, total: 19 s
Wall time: 10.2 s
Done Solving


In [5]:
"""
Obtain train error. Logloss for classification and RMSE(Root Mean Squared Error) for regression.
"""
error_train = pd.DataFrame(model.error_best,index=model.alphas)
if family == "logistic":
    print("Logloss per alpha value (-1.00 = missing)\n")
else:
    print("RMSE per alpha value (-1.00 = missing)\n")
headers = ["Alphas","Train", "CV", "Valid"]
print(tabulate(error_train,headers=headers,tablefmt="pipe", floatfmt=".2f"))

Logloss per alpha value (-1.00 = missing)

|   Alphas |   Train |   CV |   Valid |
|---------:|--------:|-----:|--------:|
|     0.00 |    0.48 | 0.48 |   -1.00 |
|     0.25 |    0.48 | 0.48 |   -1.00 |
|     0.50 |    0.48 | 0.48 |   -1.00 |
|     0.75 |    0.48 | 0.48 |   -1.00 |
|     1.00 |    0.48 | 0.48 |   -1.00 |


In [6]:
"""
Make predictions on validation set
"""
print("Predictions per alpha")
preds = model.predict(valid_x, valid_y)
print(preds)

Predictions per alpha
[[ 0.75797242  0.14470804  0.1429279  ...,  0.27083254  0.24739125
   0.17831427]
 [ 0.76288098  0.14449228  0.1463217  ...,  0.26325116  0.24426968
   0.17895791]
 [ 0.7650798   0.14582309  0.14982352 ...,  0.2563149   0.24326634
   0.18063523]
 [ 0.77001935  0.14513394  0.14949058 ...,  0.25452518  0.24248776
   0.17996302]
 [ 0.76526856  0.14233093  0.15149982 ...,  0.25497651  0.24196053
   0.1872876 ]]


In [7]:
"""
Get logloss or rmse for validation set per alpha
"""
for i in range(model.n_alphas):
    if family == "logistic":
        print("Logloss for alpha = ",model.alphas_best[i])
        print(metrics.log_loss(valid_y, preds[i]))
    else:
        print("RMSE for alpha = ",model.alphas_best[i])
        print(metrics.rmse(valid_y,preds[i]))

Logloss for alpha =  [ 0.]
0.438937751181
Logloss for alpha =  [ 0.25]
0.439073290807
Logloss for alpha =  [ 0.5]
0.439257639164
Logloss for alpha =  [ 0.75]
0.439378545941
Logloss for alpha =  [ 1.]
0.439377973888
